각 생육단계로 변화하는 날짜 계산
- mask_id 단위로 각 생육단계별 첫 등장 check_datetime 도출 -> 각 날짜 사이의 diff().day 구하기
- 결과: "mask_id", "unripe_to_semi-ripe", "semi-ripe_to_ripe"

In [51]:
import pandas as pd

In [52]:
data = pd.read_csv("merge_drowth_real_final_filtering_final.csv")

In [53]:
data.info()
data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127857 entries, 0 to 127856
Data columns (total 33 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   check_datetime     127857 non-null  object 
 1   측정일자               127857 non-null  object 
 2   구역명                127857 non-null  object 
 3   farm_code          127857 non-null  object 
 4   내부온도               126222 non-null  float64
 5   내부습도               126222 non-null  float64
 6   내부CO2              126222 non-null  float64
 7   근권온도               126222 non-null  float64
 8   내부순간일사량            126222 non-null  float64
 9   측정년도               127857 non-null  int64  
 10  측정월                127857 non-null  int64  
 11  input_image_name   127857 non-null  object 
 12  input_image_path   127857 non-null  object 
 13  img_id             127857 non-null  object 
 14  area_code          127857 non-null  object 
 15  shot_datetime      127857 non-null  object 
 16  co

,check_datetime,측정일자,구역명,farm_code,내부온도,내부습도,내부CO2,근권온도,내부순간일사량,측정년도,...,point_y,size,color,output_image_path,origin_r,origin_g,origin_b,mask_count,is_filtering,pred_growth
0,2021-09-10 13:00:00,2021-09-10,1구역,F0016,27.52,72.22,467.9,22.51,223.17,2021,...,554.744629,535.0,"[[121, 123, 112], [123, 125, 114], [118, 120, ...",image_mask_1.png,0.351564,0.406222,0.242214,2.0,0.0,unripe
1,2021-09-10 13:00:00,2021-09-10,1구역,F0016,27.52,72.22,467.9,22.51,223.17,2021,...,100.000298,37040.0,"[[54, 63, 44], [88, 97, 76], [82, 91, 70], [85...",image_mask_2.png,0.367803,0.445607,0.186590,2.0,0.0,unripe
2,2021-09-10 17:00:00,2021-09-10,1구역,F0016,27.02,71.23,463.1,23.77,102.75,2021,...,556.137573,576.0,"[[70, 71, 65], [68, 69, 63], [63, 64, 58], [49...",image_mask_1.png,0.342620,0.379780,0.277600,7.0,0.0,unripe


In [54]:
data["is_filtering"].unique() # 필터링된 결과만 존재

array([0.])

In [55]:
res_data = data.copy()

In [56]:
group_data = res_data.groupby(["mask_id", "pred_growth"])["check_datetime"].first().reset_index()

In [57]:
group_data[group_data["mask_id"] == "3eff761e4aef7098d7838dce2a29711e_1"].head(5)

,mask_id,pred_growth,check_datetime
9958,3eff761e4aef7098d7838dce2a29711e_1,ripe,2021-11-07 12:00:00
9959,3eff761e4aef7098d7838dce2a29711e_1,semi-ripe,2021-11-06 08:00:00
9960,3eff761e4aef7098d7838dce2a29711e_1,unripe,2021-11-02 13:00:00


In [58]:
group_data["check_datetime"] = pd.to_datetime(group_data["check_datetime"])

In [59]:
res = {"mask_id":[], "unripe_to_semi-ripe":[], "semi-ripe_to_ripe":[]}

for i, mask_id in enumerate(res_data["mask_id"].unique()):
    print(f"{i}번째 mask_id : {mask_id}")
    try:
        unripe_to_semiripe = abs((group_data[(group_data["mask_id"] == mask_id)&(group_data["pred_growth"] == "semi-ripe")]["check_datetime"].values[0] - \
                                group_data[(group_data["mask_id"] == mask_id)&(group_data["pred_growth"] == "unripe")]["check_datetime"].values[0]).astype('timedelta64[D]').astype(int))
    except IndexError:
        unripe_to_semiripe = None
    try:
        semiripe_to_ripe = abs((group_data[(group_data["mask_id"] == mask_id)&(group_data["pred_growth"] == "ripe")]["check_datetime"].values[0] - \
                                group_data[(group_data["mask_id"] == mask_id)&(group_data["pred_growth"] == "semi-ripe")]["check_datetime"].values[0]).astype('timedelta64[D]').astype(int))
    except IndexError:
        semiripe_to_ripe = None
    
    res["mask_id"].append(mask_id)
    res["unripe_to_semi-ripe"].append(unripe_to_semiripe)
    res["semi-ripe_to_ripe"].append(semiripe_to_ripe)

final = pd.DataFrame(res)

0번째 mask_id : 065e7a9bdd5065fe01976ff3f4f49f03_0
1번째 mask_id : c8f4ef001e5f686be96b1cb1bb68f68d_0
2번째 mask_id : 3cf79ada342fb0b16364c9ea6d703f9f_0
3번째 mask_id : 491cac2d8aeb811f039441b25b8483b0_0
4번째 mask_id : a93419f5079b7a10c49bdbefa7212931_0
5번째 mask_id : e7a51ad9cb0adeb86dbca9b32e8ae546_0
6번째 mask_id : fd48e0393a4d25876b5fb82e9571361e_0
7번째 mask_id : 3023258416cf82bfae31ce3199d34a62_0
8번째 mask_id : 66d7363516223557cf19fdbaeffd4ef1_0
9번째 mask_id : 7d78902ab2ef05dfe489937a97aa69c5_0
10번째 mask_id : 00715d5ec46b3d94a8ee5e41cf21613e_0
11번째 mask_id : 3e13eb37ab42aa111180a18790515a79_0
12번째 mask_id : 52d0cdd26abeaa1eb7d0c1dd2b02aa16_0
13번째 mask_id : b2ff0a94f4e9e1f210421f8a3275741a_0
14번째 mask_id : c6fa1a77e79158ea4befb430175636c8_0
15번째 mask_id : 57b2cac3d604c10bbfeb3e93602cbddb_0
16번째 mask_id : 6cbfc21e2b65aba2fd523ca417a174f4_0
17번째 mask_id : 68baa0aabcea125de0f2e73a970e7022_0
18번째 mask_id : e57ce9f02faa7e1128c06861f3573d09_0
19번째 mask_id : c968257436247670e60a63f2b23161f6_0
20번째 mask_

In [60]:
# + 1일 (0일이 되는 것 방지)
final["unripe_to_semi-ripe"] += 1
final["semi-ripe_to_ripe"] += 1

In [62]:
final[final["mask_id"] == "9d580a25bb5c5fd7a854f7ad09d3c13e_4"]

,mask_id,unripe_to_semi-ripe,semi-ripe_to_ripe
1900,9d580a25bb5c5fd7a854f7ad09d3c13e_4,1.0,2.0


In [63]:
# 전체 데이터와 merge
merged_final = res_data.merge(final, on="mask_id")

In [64]:
merged_final[merged_final["mask_id"]=="9d580a25bb5c5fd7a854f7ad09d3c13e_4"][["check_datetime", "pred_growth", "unripe_to_semi-ripe", "semi-ripe_to_ripe"]]

,check_datetime,pred_growth,unripe_to_semi-ripe,semi-ripe_to_ripe
10840,2022-02-25 08:30:00,unripe,1.0,2.0
10930,2022-02-25 12:50:00,unripe,1.0,2.0
11024,2022-02-25 16:40:00,semi-ripe,1.0,2.0
11112,2022-02-26 08:30:00,semi-ripe,1.0,2.0
11200,2022-02-26 12:50:00,semi-ripe,1.0,2.0
11289,2022-02-26 16:30:00,semi-ripe,1.0,2.0
11378,2022-02-27 08:30:00,semi-ripe,1.0,2.0
11473,2022-02-27 12:30:00,ripe,1.0,2.0
11575,2022-02-27 16:30:00,ripe,1.0,2.0
11668,2022-02-28 08:30:00,ripe,1.0,2.0


In [69]:
# 필터링됨
merged_final[merged_final["is_filtering"] > 0].head(3)

,check_datetime,측정일자,구역명,farm_code,내부온도,내부습도,내부CO2,근권온도,내부순간일사량,측정년도,...,color,output_image_path,origin_r,origin_g,origin_b,mask_count,is_filtering,pred_growth,unripe_to_semi-ripe,semi-ripe_to_ripe


In [72]:
merged_final.info()

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127857 entries, 0 to 127856
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   check_datetime       127857 non-null  object 
 1   측정일자                 127857 non-null  object 
 2   구역명                  127857 non-null  object 
 3   farm_code            127857 non-null  object 
 4   내부온도                 126222 non-null  float64
 5   내부습도                 126222 non-null  float64
 6   내부CO2                126222 non-null  float64
 7   근권온도                 126222 non-null  float64
 8   내부순간일사량              126222 non-null  float64
 9   측정년도                 127857 non-null  int64  
 10  측정월                  127857 non-null  int64  
 11  input_image_name     127857 non-null  object 
 12  input_image_path     127857 

In [74]:
# 저장
merged_final.sort_values(by=["farm_code", "check_datetime"], inplace=True)
merged_final.to_csv("./growth_preprocessed_filtering_data.csv", encoding="utf8", index=False)